# import data

In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta, time
stk_code = '603355'
exchange = 'sh_exchange'
folder_path = f'/Users/bokie/code/python/CMS/TWAP优化/trading-optimize/stock_data/{exchange}/{stk_code}'

files_pkl = os.listdir(folder_path)
files_pkl.remove('.DS_Store')
files_pkl
stk_dic = {}
for file in files_pkl:
    tmp_path = folder_path + '/' + file
    df_tmp = pd.read_pickle(tmp_path)
    tmp_key = file[7:15]
    stk_dic[tmp_key] = df_tmp

# date = '20220602'
# date = '20220701'
# date = '20220801'
# date = '20220812'
# date = '20220815'
# date = '20220816'
# date = '20220817'
# date = '20220818'
# date = '20220819'
# date = '20220825'
date = '20220920'
short_open_threshold = 0.785043	 #做空开仓
short_close_threshold = 0 #做空平仓

long_close_threshold = -0 #做多平仓
long_open_threshold = -1.732858 #做多开仓

# start_date = '20220819'
# end_date = '20220819'
start_date = date
end_date = date
start_date_formatted = start_date[:4] + '-' + start_date[4:6] + '-' + start_date[6:]
end_date_formatted = end_date[:4] + '-' + end_date[4:6] + '-' + end_date[6:]
# pd.set_option('display.max_rows', None)
# df_stk_orgin['index_col'] = df_stk_orgin['date_time']
# row.set_index('index_col').iloc[:,113:114]    
df_stk_orgin = stk_dic[date]

In [2]:
date[:4] + '-' + date[4:6] + '-' + date[6:]

'2022-09-20'

In [3]:
# import plotly.graph_objs as go
# df_stk_orgin['spread'] = df_stk_orgin['offer_price1'] - df_stk_orgin['bid_price1']
# fig = go.Figure(data=[
#     go.Scatter(name='log-OFI', x=df_stk_orgin['spread'].index, y=df_stk_orgin['spread']),
# ],
# )
# fig.show() 


# signal calculate

In [4]:
pd.set_option('display.max_rows', None)
cols = ['date_time', 
'bid_price1','offer_price1',
'bid_price2','offer_price2',
'bid_price3','offer_price3',
'bid_price4','offer_price4',
'bid_price5','offer_price5',
'bid_order_qty1','offer_order_qty1',
'bid_order_qty2','offer_order_qty2',
'bid_order_qty3','offer_order_qty3',
'bid_order_qty4','offer_order_qty4',
'bid_order_qty5','offer_order_qty5',
'last_px']
# df_stk_orgin[cols].head(100)

In [5]:
import numpy as np
# from sqlalchemy import false

df_signal = pd.DataFrame(data=[],columns=['wt','vt'])

pre_bid = np.average(df_stk_orgin['bid_price1'].tolist()[0]+df_stk_orgin['bid_price2'].tolist()[0]+\
                    df_stk_orgin['bid_price3'].tolist()[0]+df_stk_orgin['bid_price4'].tolist()[0]+\
                    df_stk_orgin['bid_price5'].tolist()[0])

pre_offer = np.average(df_stk_orgin['offer_price1'].tolist()[0]+df_stk_orgin['offer_price2'].tolist()[0]+\
                    df_stk_orgin['offer_price3'].tolist()[0]+df_stk_orgin['offer_price4'].tolist()[0]+\
                    df_stk_orgin['offer_price5'].tolist()[0])

pre_bid_qty = df_stk_orgin['bid_order_qty1'].tolist()[0] + df_stk_orgin['bid_order_qty2'].tolist()[0]+\
                df_stk_orgin['bid_order_qty3'].tolist()[0]+df_stk_orgin['bid_order_qty4'].tolist()[0]+\
                df_stk_orgin['bid_order_qty5'].tolist()[0]

pre_offer_qty = df_stk_orgin['offer_order_qty1'].tolist()[0] + df_stk_orgin['offer_order_qty2'].tolist()[0]+\
                df_stk_orgin['offer_order_qty3'].tolist()[0] + df_stk_orgin['offer_order_qty4'].tolist()[0]+\
                df_stk_orgin['offer_order_qty5'].tolist()[0]

pre_last_px = df_stk_orgin['last_px'].tolist()[0]

for index,row in df_stk_orgin[cols].set_index('date_time').iterrows():
    bid_qty = row['bid_order_qty1']+row['bid_order_qty2']+row['bid_order_qty3']+row['bid_order_qty4']+row['bid_order_qty5']

    offer_qty = row['offer_order_qty1']+row['offer_order_qty2']+row['offer_order_qty3']+row['offer_order_qty4']+row['offer_order_qty5']

    bid_price = np.average(row['bid_price1']+row['bid_price2']+row['bid_price3']+row['bid_price4']+row['bid_price5'])

    offer_price = np.average(row['offer_price1']+row['offer_price2']+row['offer_price3']+row['offer_price4']+row['offer_price5'])

    if bid_price > pre_bid:
        tmp_wt = np.log(bid_qty)

    # assert False
    elif bid_price == pre_bid:
        tmp_wt = np.log(bid_qty) - np.log(pre_bid_qty)
    elif bid_price < pre_bid:
        tmp_wt = -np.log(pre_bid_qty)
    
    if offer_price > pre_offer:
        tmp_vt = -np.log(pre_offer_qty)
    elif offer_price == pre_offer:
        tmp_vt = np.log(offer_qty) - np.log(pre_offer_qty)
    elif offer_price < pre_offer:
        tmp_vt = np.log(offer_qty)

    df_signal.loc[pd.to_datetime(str(index)),'wt'] = tmp_wt #原始的时间标签是int类型，先转成str再转datetime
    df_signal.loc[pd.to_datetime(str(index)),'vt'] = tmp_vt
    df_signal.loc[pd.to_datetime(str(index)),'price'] = row['last_px']
    df_signal.loc[pd.to_datetime(str(index)),'spread'] = row['offer_price1'] - row['bid_price1']
    df_signal.loc[pd.to_datetime(str(index)),'return'] = (row['last_px'] - pre_last_px)/pre_last_px
    pre_bid = bid_price
    pre_offer = offer_price
    pre_bid_qty = bid_qty
    pre_offer_qty = offer_qty
    pre_last_px = row['last_px']

df_signal['log-OFI'] = df_signal['wt'] - df_signal['vt']
df_signal['log-OFI-MA50'] = df_signal['log-OFI'].rolling(50).mean()
df_signal['log-OFI-MA100'] = df_signal['log-OFI'].rolling(100).mean()
df_signal['log-OFI-MA150'] = df_signal['log-OFI'].rolling(150).mean()
df_signal['log-OFI-MA200'] = df_signal['log-OFI'].rolling(200).mean()
df_signal['vol'] = df_signal['return'].rolling(100).std()

/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_24115/379651746.py:53: RuntimeWarning: invalid value encountered in double_scalars
  df_signal.loc[pd.to_datetime(str(index)),'return'] = (row['last_px'] - pre_last_px)/pre_last_px
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_24115/379651746.py:53: RuntimeWarning: divide by zero encountered in double_scalars
  df_signal.loc[pd.to_datetime(str(index)),'return'] = (row['last_px'] - pre_last_px)/pre_last_px
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_24115/379651746.py:47: RuntimeWarning: divide by zero encountered in log
  tmp_vt = np.log(offer_qty)
/var/folders/4y/tm0f5s2n00s3g71c6qrvd0bh0000gn/T/ipykernel_24115/379651746.py:43: RuntimeWarning: divide by zero encountered in log
  tmp_vt = -np.log(pre_offer_qty)


In [6]:
# df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()

In [7]:
incicator_stat = df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].dropna().describe()
# incicator_stat
incicator_stat['mean'] + 2*incicator_stat['std'],incicator_stat['mean'] - 2*incicator_stat['std']

(1.1474690995915717, -1.2567457343102373)

# signal check

In [24]:
import plotly.offline as pyoff
import plotly.graph_objs as go
 
x_values = df_signal.reset_index()['index'].apply(str)
# x_values = [i for i in range(len(df_signal))]
trace1 = go.Scatter(
    x=x_values,
    y=df_signal['price'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='price',
    xaxis='x', 
    yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

trace2 = go.Scatter(
    x=x_values,
    y=df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA200'
)

trace3 = go.Scatter(
    x=df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA100'
)

trace4 = go.Scatter(
    x=df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['log-OFI-MA50'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA50'
)

trace5 = go.Scatter(
    x=df_signal['spread'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['spread'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='spread'
)

trace6 = go.Scatter(
    x=df_signal['vol'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['vol'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='vol',
    # xaxis='x', 
    # yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

trace7 = go.Scatter(
    x=df_signal['return'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].index,
    y=df_signal['return'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='return',
    # xaxis='x', 
    # yaxis='y2'#标明设置一个不同于trace1的一个坐标轴
)

data = [trace1,trace2,trace3,trace4]
data = [trace1,trace2]
data = [trace1,trace3]
data = [trace1,trace5]
data = [trace1,trace6]
data = [trace1,trace7]
layout = go.Layout({"template": 'simple_white',
                   "title":{"text": 'price & indicator', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})
 
fig = go.Figure(data=data, layout=layout)
fig.show()

# data = [trace5]
# layout = go.Layout({"template": 'simple_white',
#                    "title":{"text": 'price & log-OFI-MA', "x": 0.5}, 
#                    "xaxis":{"title": {"text": None}, "tickformat": ''},
#                    "yaxis":{"title": {"text": ""}},
#                    "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
#                    "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
#                    "width": 1200,
#                    "height": 500})
 
# fig = go.Figure(data=data, layout=layout)
# fig.show()


In [9]:
from statsmodels.tsa.stattools import adfuller
if adfuller(df_signal['log-OFI-MA100'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'].dropna())[1] < 0.001:
    print("This series is stationary!")
else:
    print("This series is not stationary!")

This series is stationary!


In [10]:
# assert False

In [11]:
import plotly.graph_objects as go
 
fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=df_signal.index, y=df_signal['log-OFI']),
    
],
)

fig.show()    

In [12]:
# df_stk = df_stk_orgin.copy(deep=True)
# df_stk['date_time'] = df_stk['date_time'].apply(lambda x:pd.to_datetime(str(x)))
# df_stk = df_stk.set_index('date_time')

In [13]:
df_signal = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']
# df_stk = df_stk.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']

In [14]:
# df_position = pd.DataFrame(data=[],columns=['pos'])
# # sigma = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['std']
# # mean = df_signal.loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00']['log-OFI'].apply(float).describe()['mean']
# # short_threshold = mean + 2*sigma
# # long_threshold = mean - 2*sigma
# short_threshold = 10
# long_threshold = -10
# for index,row in df_signal.iterrows():
#     indicator = row['log-OFI-MA100']
#     if row['log-OFI'] > short_threshold:
#         df_position.loc[index,'pos'] = 1
#     elif row['log-OFI'] < long_threshold:
#         df_position.loc[index,'pos'] = -1
#     else:
#         df_position.loc[index,'pos'] = 0
#     df_position.loc[index,'price'] = row['price']

In [15]:
df_position = pd.DataFrame(data=[],columns=['pos'])

# short_open_threshold = 1 #做空开仓
# short_close_threshold = 0 #做空平仓

# long_close_threshold = 0 #做多平仓
# long_open_threshold = -1 #做多开仓
 
intraday_open_times = 0
max_open = 3

long_open_flag = 0
short_open_flag = 0

for index,row in df_signal.iterrows():
    indicator = row['log-OFI-MA200']
    if index.time() > time(10,0,0):
        if indicator >= short_open_threshold and short_open_flag==0 and long_open_flag==0:
            if intraday_open_times < max_open:
                #diff落在A区间，且无仓位，做空
                short_open_flag = 1
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator >= short_open_threshold and short_open_flag==0 and long_open_flag==1:
            if intraday_open_times < max_open:
                #diff落在A区间，且有多头仓位，平仓再做空
                short_open_flag = 1
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator >= short_open_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在A区间，且有空头仓位，keep
            short_open_flag = 1
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag


        if indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在B区间，且有空头仓位，keep
            short_open_flag = 1
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在B区间，且有多头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator > short_close_threshold and indicator < short_open_threshold and short_open_flag==0 and long_open_flag==0:
            #diff落在B区间，且无仓位，keep
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag


        if indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在C区间，且有空头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在C区间，且有多头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator >= long_close_threshold and indicator <= short_close_threshold and short_open_flag==0 and long_open_flag==0:        
            #diff落在C区间，且无仓位，keep
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag

        if indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==1 and long_open_flag==0:
            #diff落在D区间，且有空头仓位，平仓
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = -1*short_open_flag
        elif indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在D区间，且有多头仓位，keep
            short_open_flag = 0
            long_open_flag = 1
            df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator > long_open_threshold and indicator < long_close_threshold and short_open_flag==0 and long_open_flag==0:
            #diff落在D区间，且无仓位，keep
            short_open_flag = 0
            long_open_flag = 0
            df_position.loc[index,'pos'] = 1*long_open_flag

        if indicator <= long_open_threshold and short_open_flag==0 and long_open_flag==0:
            if intraday_open_times < max_open:
                #diff落在E区间，且无仓位，做多
                short_open_flag = 0
                long_open_flag = 1
                df_position.loc[index,'pos'] = 1*long_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator <= long_open_threshold and short_open_flag==1 and long_open_flag==0:
            if intraday_open_times < max_open:
                #diff落在E区间，且有空头仓位，平仓再做多
                short_open_flag = 0
                long_open_flag = 1
                df_position.loc[index,'pos'] = 1*long_open_flag
                intraday_open_times += 1
            else:
                short_open_flag = 0
                long_open_flag = 0
                df_position.loc[index,'pos'] = 1*long_open_flag
        elif indicator <= long_open_threshold and short_open_flag==0 and long_open_flag==1:
            #diff落在E区间，且有多头仓位，keep
                short_open_flag = 0
                long_open_flag = 1
                df_position.loc[index,'pos'] = 1*long_open_flag
    
    if index.time() > time(14,50,0):
        #每天14:50之后，强制平仓
        short_open_flag = 0
        long_open_flag = 0
        df_position.loc[index,'pos'] = 1*long_open_flag


df_position = pd.concat([df_position,df_signal],axis=1)
df_position.head()

,pos,wt,vt,price,spread,return,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA150,log-OFI-MA200,vol
2022-09-20 09:30:00,NaN,-0.129212,0.0,31.79,0.02,-0.000629,-0.129212,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:03,NaN,10.034516,-9.913438,31.88,0.08,0.002831,19.947954,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:06,NaN,-10.034516,9.642123,31.82,0.14,-0.001882,-19.676639,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:09,NaN,11.176053,9.487972,31.96,0.01,0.004400,1.688081,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:12,NaN,9.296518,-9.487972,32.19,0.16,0.007196,18.78449,NaN,NaN,NaN,NaN,NaN


In [16]:
# assert False

# backtest

In [17]:
df_signal_bt = df_position.iloc[0:1,:].append(df_position.dropna())
df_signal_bt.iloc[0,0] = 0 
df_signal_bt.head()

,pos,wt,vt,price,spread,return,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA150,log-OFI-MA200,vol
2022-09-20 09:30:00,0,-0.129212,0.0,31.79,0.02,-0.000629,-0.129212,NaN,NaN,NaN,NaN,NaN
2022-09-20 10:00:01,0,8.961879,0.0,31.91,0.03,-0.000313,8.961879,-0.699872,-1.452343,-0.446061,-0.496340,0.000314
2022-09-20 10:00:05,0,0.0,-0.179048,31.91,0.03,0.000000,0.179048,-0.699580,-1.367930,-0.371581,-0.494085,0.000311
2022-09-20 10:00:08,0,0.012739,-0.040005,31.91,0.03,0.000000,0.052744,-0.698129,-1.367241,-0.486283,-0.407942,0.000311
2022-09-20 10:00:11,0,8.948976,0.020203,31.91,0.02,0.000000,8.928773,-0.519217,-1.277061,-0.541402,-0.320600,0.000310


In [18]:
df_trade = pd.DataFrame([])

cash = 0 #初始账户现金
stk_hold = 0 #初始股票持有量

commission_rate = 2/10000
tax_rate = 1/1000

stk_volume = 1/3

for i in range(len(df_signal_bt)-1):

    # df_position的column
    # ['pos','wt','vt','price','log-OFI','log-OFI-MA50','log-OFI-MA100','log-OFI-MA200']
    if df_signal_bt.values[i][0] != df_signal_bt.values[i+1][0]:
        indicator = df_signal_bt.values[i+1][-1]
        print('record!',df_signal_bt.index[i+1],indicator)
        index = df_signal_bt.index[i+1]
        
        pre_pos = df_signal_bt.values[i][0]
        pos = df_signal_bt.values[i+1][0]
        
        stk_price = df_signal_bt.values[i+1][3]
        commission = commission_rate * stk_price * stk_volume
        if pre_pos == 1:
            tax = tax_rate * stk_price * stk_volume
        else:
            tax = 0

        if pos == 0 and pre_pos == 1:
            cash = cash + stk_price*stk_volume - commission - tax
            stk_hold = 0
            asset_value = 0
        elif pos == 0 and pre_pos == -1:
            cash = cash - stk_price*stk_volume - commission - tax
            stk_hold = 0
            asset_value = 0
        
        elif pos == 1 and pre_pos == 0:
            cash = cash - stk_price*stk_volume - commission - tax
            stk_hold = 1*stk_volume
            asset_value = stk_hold*stk_price
        elif pos == 1 and pre_pos == -1 :
            cash = cash + (stk_price*stk_volume - commission - tax)*2
            stk_hold = 1*stk_volume
            asset_value = stk_hold*stk_price
        
        elif pos == -1 and pre_pos == 0:
            cash = cash + stk_price*stk_volume - commission - tax
            stk_hold = -1*stk_volume
            asset_value = stk_hold*stk_price
        elif pos == -1 and pre_pos == 1:
            cash = cash + (stk_price*stk_volume - commission - tax)*2
            stk_hold = -1*stk_volume
            asset_value = stk_hold*stk_price
        
            
    ########################################################################################################################
    # asset_value = stk_hold*stk_price
        df_trade.loc[index,'log-OFI-MA200'] = np.round(indicator,4)
        df_trade.loc[index,'pos'] = pos
        df_trade.loc[index,'stock hold'] = stk_hold
        df_trade.loc[index,'price'] = stk_price
        df_trade.loc[index,'asset value'] = asset_value
        df_trade.loc[index,'cash'] = cash
        df_trade.loc[index,'commission'] = commission
        df_trade.loc[index,'tax'] = tax
    # df_trade.loc[index,'net value'] = cash + asset_value
df_trade['net value'] = df_trade['asset value'] + df_trade['cash']
# df_trade['long/short threshold'] = [str(long_threshold)+'@'+str(short_threshold)]*len(df_trade)

record! 2022-09-20 10:20:16 0.00032579601288649785
record! 2022-09-20 10:31:49 0.00031533272043152323
record! 2022-09-20 10:37:52 0.0005645010820849545
record! 2022-09-20 10:49:57 0.0008425916229547068
record! 2022-09-20 10:59:28 0.0003377777563456421
record! 2022-09-20 11:02:10 0.000799652870506013


In [19]:
df_position

,pos,wt,vt,price,spread,return,log-OFI,log-OFI-MA50,log-OFI-MA100,log-OFI-MA150,log-OFI-MA200,vol
2022-09-20 09:30:00,NaN,-0.129212,0.0,31.79,0.02,-0.000629,-0.129212,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:03,NaN,10.034516,-9.913438,31.88,0.08,0.002831,19.947954,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:06,NaN,-10.034516,9.642123,31.82,0.14,-0.001882,-19.676639,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:09,NaN,11.176053,9.487972,31.96,0.01,0.004400,1.688081,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:12,NaN,9.296518,-9.487972,32.19,0.16,0.007196,18.78449,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:14,NaN,8.853665,-8.716044,32.19,0.18,0.000000,17.569709,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:17,NaN,-8.853665,8.594154,31.99,0.20,-0.006213,-17.44782,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:20,NaN,9.615805,-0.160343,32.11,0.17,0.003751,9.776148,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:23,NaN,-9.615805,8.665613,32.00,0.04,-0.003426,-18.281419,NaN,NaN,NaN,NaN,NaN
2022-09-20 09:30:26,NaN,8.49699,-8.665613,32.19,0.15,0.005937,17.162604,NaN,NaN,NaN,NaN,NaN


In [20]:
df_trade

,log-OFI-MA200,pos,stock hold,price,asset value,cash,commission,tax,net value
2022-09-20 10:20:16,0.0003,-1.0,-0.333333,32.16,-10.720000,10.717856,0.002144,0.0,-0.002144
2022-09-20 10:31:49,0.0003,0.0,0.000000,32.10,0.000000,0.015716,0.002140,0.0,0.015716
2022-09-20 10:37:52,0.0006,-1.0,-0.333333,32.30,-10.766667,10.780229,0.002153,0.0,0.013563
2022-09-20 10:49:57,0.0008,0.0,0.000000,32.56,0.000000,-0.075275,0.002171,0.0,-0.075275
2022-09-20 10:59:28,0.0003,-1.0,-0.333333,32.64,-10.880000,10.802549,0.002176,0.0,-0.077451
2022-09-20 11:02:10,0.0008,0.0,0.000000,32.62,0.000000,-0.072959,0.002175,0.0,-0.072959


In [21]:
df_trade_long = df_trade[df_trade['pos']==1]
df_trade_short = df_trade[df_trade['pos']==-1]
df_trade_close = df_trade[df_trade['pos']==0]

# performance analysis

In [22]:
x_values = df_trade['net value'].index
# x_values = [i for i in range(len(df_trade))]

fig = go.Figure(data=[
    go.Scatter(name='log-OFI', x=x_values, y=df_trade['net value']),
],
)
fig.show() 

In [23]:
'''
rgba颜色对照表
https://wenku.baidu.com/view/41c91c5b24284b73f242336c1eb91a37f0113257.html
plotly绘制散点图
https://www.yisu.com/zixun/168081.html
'''

x_values = df_signal.reset_index()['index'].apply(str)
# x_values = [i for i in range(len(df_signal))]
trace1 = go.Scatter(
    x=x_values,
    y=df_signal['price'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='price',
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    mode = 'lines'
)

trace2 = go.Scatter(
    x=x_values,
    y=df_signal['log-OFI-MA200'].loc[f'{start_date_formatted} 09:30:00':f'{end_date_formatted} 14:55:00'],
    name='log-OFI-MA200',
    mode = 'lines'
)


trace_long = go.Scatter(
    x = df_trade_long.index,
    y = df_trade_long['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'long open',
    mode = 'markers',
    marker = dict(
    size = 8, # 设置点的宽度
    color = 'rgba(227, 23, 13, .9)'), #红色
    )

trace_short = go.Scatter(
    x = df_trade_short.index,
    y = df_trade_short['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'short open',
    mode = 'markers',
    marker = dict(
    size =8, # 设置点的宽度
    color = 'rgba(151, 51, 250, .9)'), #紫色
    )

trace_close = go.Scatter(
    x = df_trade_close.index,
    y = df_trade_close['price'],
    xaxis='x', 
    yaxis='y2',#标明设置一个不同于trace1的一个坐标轴
    name = 'close',
    mode = 'markers',
    marker = dict(
    size = 8, # 设置点的宽度
    color = 'rgba(0, 255, 0, .9)'), #绿色
    )

data = [trace1,trace2,trace_long,trace_short,trace_close]
layout = go.Layout({"template": 'simple_white',
                   "title":{"text": 'price & log-OFI-MA', "x": 0.5}, 
                   "xaxis":{"title": {"text": None}, "tickformat": ''},
                   "yaxis":{"title": {"text": ""}},
                   "yaxis2": {'anchor': 'x', "overlaying": 'y', "side": 'right'}, #设置坐标轴的格式，一般次坐标轴在右侧
                   "legend":{"title": {"text": ""}, "x": 0.9, "y": 1.1},
                   "width": 1200,
                   "height": 500})

fig = go.Figure(data=data, layout=layout)
fig.update_layout(legend=dict(
    yanchor="top",
    # y=0.99,
    xanchor="center",
    x=0.5,
    # bgcolor="LightSteelBlue",
    # bordercolor="Black",
))
fig.show()
